<a href="https://colab.research.google.com/github/MatheusHenriqueDaSilva/data_science/blob/main/Exercicio_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split


In [29]:
# Data understanding
DATA = pd.read_csv('/content/classificacao_1.csv')
DATA.head()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,v_15,v_16,v_17,v_18,target
0,-0.79207,-1.53485,0.60615,-2.27976,-1.43551,2.44272,-1.18421,1.83623,-2.79111,0.38763,-2.84928,-1.45238,-3.43264,-0.70667,-0.71361,-2.32598,-2.71869,-1.76932,0
1,0.21583,-0.93657,0.75756,0.37746,-2.60988,1.85922,2.62460,0.24354,-3.43729,-0.26536,1.34649,-1.45203,3.12942,2.18204,-0.95989,-2.69853,4.91233,0.09381,0
2,-0.32059,-2.16565,-1.63757,0.79361,-1.95570,-0.18525,-0.88095,-0.98657,-4.14707,1.61340,-0.68718,-1.17566,-0.04852,1.20091,1.55407,-3.31448,0.24915,3.10828,0
3,-0.12929,-0.06134,3.58236,-1.48172,1.41317,-1.47032,-1.05289,1.87866,-1.99177,1.13534,0.49201,-0.21985,-1.43865,-0.64562,-0.28754,3.76759,-3.87927,-0.74953,0
4,-1.85108,-3.65335,0.10311,-0.18259,-1.71422,1.21289,-2.22696,-1.53030,-2.15125,0.77353,-2.36629,-1.76121,-5.21324,-0.88661,-0.24870,0.90993,-0.10623,-1.38851,0


In [30]:
def generate_metadata(dataframe):

    metadata2 = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata2=metadata2.sort_values(by='tipo')
    metadata2 = metadata2.reset_index(drop=True)

    return metadata2

metadados = generate_metadata(DATA)
metadados

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,target,int64,0,0.0,2
1,v_17,float64,0,0.0,1249
2,v_16,float64,0,0.0,1249
3,v_15,float64,0,0.0,1246
4,v_14,float64,0,0.0,1248
5,v_13,float64,0,0.0,1248
6,v_12,float64,0,0.0,1249
7,v_11,float64,0,0.0,1249
8,v_18,float64,0,0.0,1249
9,v_1,float64,0,0.0,1250


In [31]:
DATA.describe()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,v_15,v_16,v_17,v_18,target
count,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000
mean,-0.488123,-0.507797,1.036540,-0.400280,-0.570370,0.486453,-0.401489,0.437645,-0.541071,0.011601,0.465734,-0.468205,-0.777670,-1.537798,-0.007331,0.406387,-0.990709,-0.090687,0.521600
std,2.203388,2.170907,2.055162,2.294943,2.193388,2.247761,2.385788,2.228049,2.099253,1.025620,2.102606,1.942553,4.013094,4.226739,1.019357,4.396981,1.992697,2.310923,0.499733
min,-6.458820,-8.228040,-5.271280,-7.565150,-8.771000,-5.721300,-9.764920,-6.497230,-6.607440,-3.408470,-5.836500,-5.872420,-14.619420,-17.762630,-2.852980,-11.857600,-7.415950,-8.198030,0.000000
25%,-1.979912,-2.033467,-0.367510,-2.096425,-2.008675,-1.072315,-1.990500,-1.062978,-1.904840,-0.687910,-0.890665,-1.742657,-2.870810,-4.075943,-0.707782,-2.571003,-2.247897,-1.589525,0.000000
50%,-0.577435,-0.423765,1.024450,-0.337500,-0.554445,0.390580,-0.448425,0.382730,-0.608385,-0.001715,0.505450,-0.520500,-0.561670,-1.608000,0.008110,0.262110,-0.962245,-0.269250,1.000000
75%,0.770713,0.951267,2.373443,1.241965,0.882300,1.974765,1.089685,1.923013,0.845130,0.711928,1.823472,0.773675,1.650690,1.157640,0.642238,3.012385,0.384700,1.431752,1.000000
max,9.040990,6.386520,8.000620,6.496730,6.051110,8.500250,9.463980,8.623210,7.809400,3.145010,7.616810,6.530610,13.832710,12.033330,2.956530,16.128980,6.109730,8.408000,1.000000


In [79]:
# criando o melhor preditor de light GBM
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVC

x_treino_final = DATA.drop('target', axis=1)
y_treino_final = DATA['target']


# Definindo o modelo
model = SVC()

# Definindo os parâmetros para o grid search
param_grid = {
    'C': [1],
    'kernel': ['linear'],
    # 'degree': [2, 3],  # Só é relevante se kernel='poly'
    #'gamma': [0.1, 1]
}

# Definindo o objeto GridSearchCV
grid = GridSearchCV(model,
                    param_grid,
                    cv=KFold(n_splits = 10, shuffle=False), # padrão de shuffle é False
                    verbose=2,
                    scoring='roc_auc',  # 'accuracy' 'precision' 'recall' 'roc_auc' 'f1'
                    n_jobs=-1,
                    return_train_score = True)

# Treinando o modelo com o grid search
grid.fit(x_treino_final, y_treino_final)

# Exibindo os melhores parâmetros encontrados pelo grid search
print("Melhores Parâmetros: ", grid.best_params_)

# Exibindo a melhor pontuação (score) atingida pelo modelo com os melhores parâmetros
print("Melhor f1: ", grid.best_score_)




best_model = grid.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Melhores Parâmetros:  {'C': 1, 'kernel': 'linear'}
Melhor f1:  0.872401528850818


In [80]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix,f1_score, classification_report


predict = best_model.predict(x_treino_final)

print(confusion_matrix(y_treino_final, predict))
print(f1_score(y_treino_final, predict))
print(classification_report(y_treino_final, predict))

[[493 105]
 [123 529]]
0.8227060653188181
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       598
           1       0.83      0.81      0.82       652

    accuracy                           0.82      1250
   macro avg       0.82      0.82      0.82      1250
weighted avg       0.82      0.82      0.82      1250



In [81]:
estudo = pd.DataFrame(grid.cv_results_ )
estudo['mean_train_score']

0    0.880261
Name: mean_train_score, dtype: float64

In [82]:
estudo['mean_test_score']

0    0.872402
Name: mean_test_score, dtype: float64